# 👁️ **Módulo 4: Observabilidade e Monitoramento**

> **Tá, mas o que é Observabilidade?** É como ter **olhos em todos os lugares** do seu sistema de IA! Imagine que você é um detetive que precisa monitorar uma cidade inteira - você precisa de câmeras, sensores, alarmes e um sistema central que te avise quando algo suspeito acontece.

---

## **Aula 4.1: O que é Observabilidade em IA?**

### **Tá, mas como funciona isso?**

Observabilidade é como ter um **sistema nervoso digital** para sua IA. Em vez de só descobrir que algo deu errado quando o usuário reclama, você consegue ver **em tempo real** o que está acontecendo dentro do sistema.

**Analogia do dia a dia:**
Imagine que você é um médico monitorando um paciente em UTI. Você tem monitores que mostram batimentos cardíacos, pressão arterial, temperatura, etc. Se algo der errado, você é alertado imediatamente, não quando o paciente já está em parada cardíaca!

**Por que isso é importante?**

Sem observabilidade, você só descobre problemas quando:
- Usuários reclamam que a IA está dando respostas estranhas
- O sistema para de funcionar completamente
- Hackers já invadiram e roubaram dados
- A performance caiu drasticamente

**Com observabilidade, você descobre problemas:**
- Antes que afetem os usuários
- Em tempo real
- Com detalhes completos do que aconteceu
- Com alertas automáticos

---

**🖼️ Sugestão de imagem**: Um dashboard com múltiplos monitores mostrando métricas de IA em tempo real

## **🔧 Setup Inicial - Preparando o Terreno**

Primeiro, vamos configurar nosso ambiente de observabilidade:

In [ ]:
# 🔧 SETUP INICIAL - OBSERVABILIDADE
import os
import re
import json
import logging
import random
import time
import threading
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
import psutil
from prometheus_client import Counter, Histogram, Gauge, start_http_server

# Configurando logging avançado
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('observabilidade.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Configurando estilo dos gráficos
plt.style.use('default')
sns.set_palette("husl")

print("👁️ Ambiente de Observabilidade configurado!")
print("📝 Logs serão salvos em 'observabilidade.log'")

## ** Configurando o LLM**

Vamos configurar nosso LLM para testar técnicas de observabilidade:

In [ ]:
# CONFIGURANDO O LLM
from dotenv import load_dotenv
load_dotenv()

def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print(" Usando OpenAI GPT-3.5-turbo")
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except Exception as e:
        print(f"⚠️  OpenAI não disponível: {e}")
    
    # Tentativa 2: Hugging Face
    try:
        from langchain_community.llms import HuggingFaceHub
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            print("🤗 Usando Hugging Face (google/flan-t5-base)")
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except Exception as e:
        print(f"⚠️  Hugging Face não disponível: {e}")
    
    # Fallback: Simulação
    print("🔄 Usando modo simulação (sem API)")
    return None

# Configurando o LLM
llm = get_llm_colab()

if llm:
    print("✅ LLM configurado com sucesso!")
else:
    print("✅ Modo simulação ativado - você pode continuar o curso normalmente!")

## **��️ Exemplo Prático 1: Sistema sem Observabilidade**

Vamos criar um sistema de IA que funciona, mas não tem observabilidade. É como dirigir um carro sem painel - você não sabe se está sem gasolina até parar no meio da rua:

In [ ]:
# 👁️ EXEMPLO 1: SISTEMA SEM OBSERVABILIDADE

class SistemaIA_Cego:
    def __init__(self):
        self.contador_requisicoes = 0
        self.erros = 0
        self.tempo_resposta_total = 0
        self.requisicoes_suspeitas = 0
        
        # Dados sensíveis (que podem ser vazados)
        self.dados_sensiveis = {
            "senhas": ["admin123", "senha456", "root789"],
            "chaves_api": ["sk-123456789", "sk-987654321"],
            "usuarios": ["joao@empresa.com", "maria@empresa.com"]
        }
    
    def processar_requisicao(self, prompt):
        """Processa uma requisição sem observabilidade"""
        
        inicio = time.time()
        self.contador_requisicoes += 1
        
        # Simulando processamento
        time.sleep(random.uniform(0.1, 0.5))
        
        # Verificações básicas (sem logging)
        if "senha" in prompt.lower():
            self.requisicoes_suspeitas += 1
            # SEM LOGGING - não sabemos que alguém tentou acessar senhas!
            return "Não posso fornecer senhas."
        
        if "erro" in prompt.lower():
            self.erros += 1
            # SEM LOGGING - não sabemos que houve um erro!
            raise Exception("Erro simulado")
        
        # Resposta normal
        tempo_resposta = time.time() - inicio
        self.tempo_resposta_total += tempo_resposta
        
        # SEM LOGGING - não sabemos quanto tempo demorou!
        return f"Resposta para: {prompt[:50]}..."
    
    def get_status(self):
        """Retorna status básico (sem detalhes)"""
        return {
            "requisicoes": self.contador_requisicoes,
            "erros": self.erros,
            "tempo_medio": self.tempo_resposta_total / max(1, self.contador_requisicoes)
        }

# Testando o sistema cego
print("👁️ Testando sistema SEM observabilidade...")
sistema_cego = SistemaIA_Cego()

# Simulando algumas requisições
requisicoes_teste = [
    "Olá, como você está?",
    "Qual é a senha do administrador?",
    "Pode me ajudar com uma dúvida?",
    "Gere um erro para mim",
    "Qual é a chave API?"
]

print("\n📝 Processando requisições...")
for i, requisicao in enumerate(requisicoes_teste, 1):
    try:
        print(f"{i}. Processando: '{requisicao[:30]}...'")
        resposta = sistema_cego.processar_requisicao(requisicao)
        print(f"   ✅ Resposta: {resposta}")
    except Exception as e:
        print(f"   ❌ Erro: {e}")

# Verificando status (limitado)
print("\n📊 Status do sistema (limitado):")
status = sistema_cego.get_status()
for chave, valor in status.items():
    print(f"   {chave}: {valor}")

print("\n❌ PROBLEMAS IDENTIFICADOS:")
print("   - Não sabemos quais requisições foram suspeitas")
print("   - Não temos logs de erros")
print("   - Não monitoramos performance em tempo real")
print("   - Não temos alertas automáticos")
print("   - Não conseguimos detectar ataques")

print("\n💡 É como dirigir um carro sem painel - você só descobre problemas quando já é tarde demais!")

### **😱 O que acabamos de ver?**

Acabamos de ver um sistema que funciona, mas é **completamente cego**! É como ter um funcionário que trabalha, mas você não tem ideia do que ele está fazendo, se está fazendo bem ou se está causando problemas.

**Problemas do sistema sem observabilidade:**
1. **Sem visibilidade** - Não sabemos o que está acontecendo
2. **Sem alertas** - Só descobrimos problemas quando é tarde
3. **Sem métricas** - Não conseguimos medir performance
4. **Sem logs** - Não temos histórico do que aconteceu
5. **Sem segurança** - Não detectamos ataques

---

**💡 Dica do Professor**: Sem observabilidade, você está voando às cegas! É como tentar consertar um carro sem saber qual é o problema.

## **��️ Exemplo Prático 2: Sistema com Observabilidade Completa**

Agora vamos criar um sistema com observabilidade completa. É como ter um painel de controle de nave espacial com centenas de sensores:

In [ ]:
# 👁️ EXEMPLO 2: SISTEMA COM OBSERVABILIDADE COMPLETA

class SistemaIA_Observavel:
    def __init__(self):
        # Métricas de Prometheus
        self.requisicoes_total = Counter('requisicoes_total', 'Total de requisições')
        self.requisicoes_suspeitas = Counter('requisicoes_suspeitas', 'Requisições suspeitas')
        self.erros_total = Counter('erros_total', 'Total de erros')
        self.tempo_resposta = Histogram('tempo_resposta', 'Tempo de resposta')
        self.requisicoes_ativas = Gauge('requisicoes_ativas', 'Requisições ativas')
        
        # Logs estruturados
        self.logger = logging.getLogger('sistema_ia')
        
        # Dados sensíveis
        self.dados_sensiveis = {
            "senhas": ["admin123", "senha456", "root789"],
            "chaves_api": ["sk-123456789", "sk-987654321"],
            "usuarios": ["joao@empresa.com", "maria@empresa.com"]
        }
        
        # Cache de métricas em tempo real
        self.metricas_tempo_real = {
            'requisicoes_por_minuto': deque(maxlen=60),
            'erros_por_minuto': deque(maxlen=60),
            'tempo_medio_resposta': deque(maxlen=100),
            'requisicoes_suspeitas_por_minuto': deque(maxlen=60)
        }
        
        # Alertas
        self.alertas_ativos = []
        self.limites_alertas = {
            'erros_por_minuto': 5,
            'tempo_resposta_medio': 2.0,
            'requisicoes_suspeitas_por_minuto': 3
        }
    
    def processar_requisicao_observavel(self, prompt, usuario=None, ip=None):
        """Processa uma requisição com observabilidade completa"""
        
        # Início da requisição
        inicio = time.time()
        self.requisicoes_total.inc()
        self.requisicoes_ativas.inc()
        
        # Log estruturado da requisição
        log_data = {
            'timestamp': datetime.now().isoformat(),
            'usuario': usuario,
            'ip': ip,
            'prompt': prompt[:100],  # Limitando para não logar dados sensíveis
            'tipo': 'requisicao_inicio'
        }
        self.logger.info(f"Requisição iniciada: {json.dumps(log_data)}")
        
        try:
            # Verificações de segurança
            alertas_seguranca = self.verificar_seguranca(prompt, usuario, ip)
            if alertas_seguranca:
                for alerta in alertas_seguranca:
                    self.gerar_alerta(alerta)
            
            # Simulando processamento
            time.sleep(random.uniform(0.1, 0.5))
            
            # Verificações específicas
            if "senha" in prompt.lower():
                self.requisicoes_suspeitas.inc()
                self.logger.warning(f"Tentativa de acesso a senhas detectada: {json.dumps(log_data)}")
                return "Não posso fornecer senhas."
            
            if "erro" in prompt.lower():
                self.erros_total.inc()
                self.logger.error(f"Erro simulado gerado: {json.dumps(log_data)}")
                raise Exception("Erro simulado")
            
            # Resposta normal
            resposta = f"Resposta para: {prompt[:50]}..."
            
            # Final da requisição
            tempo_resposta = time.time() - inicio
            self.tempo_resposta.observe(tempo_resposta)
            self.requisicoes_ativas.dec()
            
            # Atualizando métricas em tempo real
            self.atualizar_metricas_tempo_real(tempo_resposta, False, False)
            
            # Log de sucesso
            log_data['tipo'] = 'requisicao_sucesso'
            log_data['tempo_resposta'] = tempo_resposta
            self.logger.info(f"Requisição concluída: {json.dumps(log_data)}")
            
            return resposta
            
        except Exception as e:
            # Tratamento de erro
            self.erros_total.inc()
            self.requisicoes_ativas.dec()
            
            tempo_resposta = time.time() - inicio
            self.atualizar_metricas_tempo_real(tempo_resposta, True, False)
            
            # Log de erro
            log_data['tipo'] = 'requisicao_erro'
            log_data['erro'] = str(e)
            log_data['tempo_resposta'] = tempo_resposta
            self.logger.error(f"Erro na requisição: {json.dumps(log_data)}")
            
            raise e
    
    def verificar_seguranca(self, prompt, usuario, ip):
        """Verifica aspectos de segurança da requisição"""
        alertas = []
        
        # Verificar palavras suspeitas
        palavras_suspeitas = ["senha", "password", "hack", "exploit", "inject"]
        if any(palavra in prompt.lower() for palavra in palavras_suspeitas):
            alertas.append({
                'tipo': 'palavras_suspeitas',
                'severidade': 'medio',
                'descricao': f"Palavras suspeitas detectadas em prompt de {usuario}"
            })
        
        # Verificar IP suspeito (simulado)
        if ip and ip.startswith('192.168.1.100'):  # IP suspeito simulado
            alertas.append({
                'tipo': 'ip_suspeito',
                'severidade': 'alto',
                'descricao': f"IP suspeito detectado: {ip}"
            })
        
        return alertas
    
    def gerar_alerta(self, alerta):
        """Gera um alerta de segurança"""
        alerta['timestamp'] = datetime.now().isoformat()
        self.alertas_ativos.append(alerta)
        
        self.logger.warning(f"ALERTA GERADO: {json.dumps(alerta)}")
        
        # Em um sistema real, aqui enviaria email, Slack, etc.
        print(f"🚨 ALERTA: {alerta['descricao']} (Severidade: {alerta['severidade']})")
    
    def atualizar_metricas_tempo_real(self, tempo_resposta, foi_erro, foi_suspeito):
        """Atualiza métricas em tempo real"""
        timestamp = time.time()
        
        # Requisições por minuto
        self.metricas_tempo_real['requisicoes_por_minuto'].append(timestamp)
        
        # Erros por minuto
        if foi_erro:
            self.metricas_tempo_real['erros_por_minuto'].append(timestamp)
        
        # Tempo médio de resposta
        self.metricas_tempo_real['tempo_medio_resposta'].append(tempo_resposta)
        
        # Requisições suspeitas por minuto
        if foi_suspeito:
            self.metricas_tempo_real['requisicoes_suspeitas_por_minuto'].append(timestamp)
    
    def get_metricas_tempo_real(self):
        """Retorna métricas em tempo real"""
        agora = time.time()
        um_minuto_atras = agora - 60
        
        return {
            'requisicoes_por_minuto': len([t for t in self.metricas_tempo_real['requisicoes_por_minuto'] if t > um_minuto_atras]),
            'erros_por_minuto': len([t for t in self.metricas_tempo_real['erros_por_minuto'] if t > um_minuto_atras]),
            'tempo_medio_resposta': np.mean(list(self.metricas_tempo_real['tempo_medio_resposta'])) if self.metricas_tempo_real['tempo_medio_resposta'] else 0,
            'requisicoes_suspeitas_por_minuto': len([t for t in self.metricas_tempo_real['requisicoes_suspeitas_por_minuto'] if t > um_minuto_atras])
        }
    
    def verificar_alertas(self):
        """Verifica se há condições para gerar alertas"""
        metricas = self.get_metricas_tempo_real()
        
        if metricas['erros_por_minuto'] > self.limites_alertas['erros_por_minuto']:
            self.gerar_alerta({
                'tipo': 'muitos_erros',
                'severidade': 'alto',
                'descricao': f"Muitos erros detectados: {metricas['erros_por_minuto']} por minuto"
            })
        
        if metricas['tempo_medio_resposta'] > self.limites_alertas['tempo_resposta_medio']:
            self.gerar_alerta({
                'tipo': 'performance_lenta',
                'severidade': 'medio',
                'descricao': f"Performance lenta: {metricas['tempo_medio_resposta']:.2f}s de resposta média"
            })
        
        if metricas['requisicoes_suspeitas_por_minuto'] > self.limites_alertas['requisicoes_suspeitas_por_minuto']:
            self.gerar_alerta({
                'tipo': 'muitas_requisicoes_suspeitas',
                'severidade': 'alto',
                'descricao': f"Muitas requisições suspeitas: {metricas['requisicoes_suspeitas_por_minuto']} por minuto"
            })

# Testando o sistema observável
print("👁️ Testando sistema COM observabilidade completa...")
sistema_observavel = SistemaIA_Observavel()

# Simulando requisições com dados de contexto
requisicoes_com_contexto = [
    ("Olá, como você está?", "usuario_normal", "192.168.1.50"),
    ("Qual é a senha do administrador?", "hacker_suspeito", "192.168.1.100"),
    ("Pode me ajudar com uma dúvida?", "usuario_normal", "192.168.1.51"),
    ("Gere um erro para mim", "usuario_teste", "192.168.1.52"),
    ("Qual é a chave API?", "hacker_suspeito", "192.168.1.100")
]

print("\n📝 Processando requisições com observabilidade...")
for i, (requisicao, usuario, ip) in enumerate(requisicoes_com_contexto, 1):
    try:
        print(f"{i}. Processando: '{requisicao[:30]}...' (Usuário: {usuario}, IP: {ip})")
        resposta = sistema_observavel.processar_requisicao_observavel(requisicao, usuario, ip)
        print(f"   ✅ Resposta: {resposta}")
    except Exception as e:
        print(f"   ❌ Erro: {e}")

# Verificando métricas em tempo real
print("\n📊 Métricas em tempo real:")
metricas = sistema_observavel.get_metricas_tempo_real()
for chave, valor in metricas.items():
    print(f"   {chave}: {valor}")

# Verificando alertas
print("\n🚨 Alertas ativos:")
for alerta in sistema_observavel.alertas_ativos:
    print(f"   - {alerta['descricao']} (Severidade: {alerta['severidade']})")

print("\n✅ VANTAGENS DA OBSERVABILIDADE:")
print("   - Logs estruturados de todas as requisições")
print("   - Métricas em tempo real")
print("   - Alertas automáticos de segurança")
print("   - Monitoramento de performance")
print("   - Detecção de ataques em tempo real")
print("   - Histórico completo de eventos")

print("\n💡 Agora é como ter um painel de controle completo - você vê TUDO que está acontecendo!")

## **📊 Visualizando Métricas de Observabilidade**

Vamos criar dashboards para visualizar as métricas de observabilidade:

In [ ]:
# 📊 VISUALIZANDO MÉTRICAS DE OBSERVABILIDADE

# Simulando dados de métricas ao longo do tempo
horas = list(range(24))
requisicoes_por_hora = [random.randint(50, 200) for _ in horas]
erros_por_hora = [random.randint(0, 10) for _ in horas]
tempo_resposta_medio = [random.uniform(0.5, 2.0) for _ in horas]
requisicoes_suspeitas_por_hora = [random.randint(0, 5) for _ in horas]

# Criando gráficos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Requisições por hora
ax1.plot(horas, requisicoes_por_hora, 'o-', color='#4ecdc4', linewidth=2, markersize=6)
ax1.set_title('Requisições por Hora', fontsize=14, fontweight='bold')
ax1.set_xlabel('Hora do Dia')
ax1.set_ylabel('Número de Requisições')
ax1.grid(True, alpha=0.3)
ax1.fill_between(horas, requisicoes_por_hora, alpha=0.3, color='#4ecdc4')

# Gráfico 2: Erros por hora
ax2.bar(horas, erros_por_hora, color='#ff6b6b', alpha=0.8)
ax2.set_title('Erros por Hora', fontsize=14, fontweight='bold')
ax2.set_xlabel('Hora do Dia')
ax2.set_ylabel('Número de Erros')
ax2.grid(True, alpha=0.3)

# Adicionando linha de alerta
ax2.axhline(y=5, color='red', linestyle='--', alpha=0.7, label='Limite de Alerta')
ax2.legend()

# Gráfico 3: Tempo de resposta médio
ax3.plot(horas, tempo_resposta_medio, 'o-', color='#feca57', linewidth=2, markersize=6)
ax3.set_title('Tempo de Resposta Médio', fontsize=14, fontweight='bold')
ax3.set_xlabel('Hora do Dia')
ax3.set_ylabel('Tempo (segundos)')
ax3.grid(True, alpha=0.3)

# Adicionando linha de alerta
ax3.axhline(y=1.5, color='orange', linestyle='--', alpha=0.7, label='Limite de Alerta')
ax3.legend()

# Gráfico 4: Requisições suspeitas
ax4.bar(horas, requisicoes_suspeitas_por_hora, color='#ff9ff3', alpha=0.8)
ax4.set_title('Requisições Suspeitas por Hora', fontsize=14, fontweight='bold')
ax4.set_xlabel('Hora do Dia')
ax4.set_ylabel('Número de Requisições Suspeitas')
ax4.grid(True, alpha=0.3)

# Adicionando linha de alerta
ax4.axhline(y=3, color='purple', linestyle='--', alpha=0.7, label='Limite de Alerta')
ax4.legend()

plt.tight_layout()
plt.show()

# Calculando estatísticas
print("📊 ESTATÍSTICAS DE OBSERVABILIDADE:")
print("=" * 50)
print(f"📈 Total de requisições (24h): {sum(requisicoes_por_hora):,}")
print(f"❌ Total de erros (24h): {sum(erros_por_hora)}")
print(f"📊 Taxa de erro: {(sum(erros_por_hora)/sum(requisicoes_por_hora)*100):.2f}%")
print(f"⏱️ Tempo de resposta médio: {np.mean(tempo_resposta_medio):.2f}s")
print(f"🚨 Total de requisições suspeitas: {sum(requisicoes_suspeitas_por_hora)}")
print(f"📈 Pico de requisições: {max(requisicoes_por_hora)} (hora {horas[requisicoes_por_hora.index(max(requisicoes_por_hora))]})")

# Análise de padrões
print(f"\n�� ANÁLISE DE PADRÕES:")
if max(requisicoes_por_hora) > 150:
    print("   ⚠️  Pico de tráfego detectado - possível ataque DDoS")
if sum(erros_por_hora) > 50:
    print("   🚨 Muitos erros detectados - possível problema no sistema")
if np.mean(tempo_resposta_medio) > 1.5:
    print("   🐌 Performance lenta detectada - otimização necessária")
if sum(requisicoes_suspeitas_por_hora) > 20:
    print("   �� Muitas requisições suspeitas - possível ataque")

print("\n✅ Sistema de observabilidade funcionando perfeitamente!")

## **🔍 Teste Rápido - Identificando Problemas com Observabilidade**

Agora é sua vez! Vamos testar se você consegue identificar problemas usando dados de observabilidade:

In [ ]:
# 🔍 TESTE RÁPIDO - IDENTIFICANDO PROBLEMAS COM OBSERVABILIDADE

def testar_analise_observabilidade():
    """Testa a capacidade de analisar dados de observabilidade"""
    
    # Cenários de problemas
    cenarios = [
        {
            'nome': 'Ataque DDoS',
            'requisicoes_por_minuto': 500,
            'erros_por_minuto': 2,
            'tempo_resposta_medio': 0.8,
            'requisicoes_suspeitas': 1,
            'descricao': 'Muitas requisições normais, poucos erros'
        },
        {
            'nome': 'Bug no Sistema',
            'requisicoes_por_minuto': 100,
            'erros_por_minuto': 25,
            'tempo_resposta_medio': 0.5,
            'requisicoes_suspeitas': 0,
            'descricao': 'Poucas requisições, muitos erros'
        },
        {
            'nome': 'Performance Lenta',
            'requisicoes_por_minuto': 80,
            'erros_por_minuto': 3,
            'tempo_resposta_medio': 3.5,
            'requisicoes_suspeitas': 2,
            'descricao': 'Tempo de resposta muito alto'
        },
        {
            'nome': 'Ataque de Segurança',
            'requisicoes_por_minuto': 120,
            'erros_por_minuto': 5,
            'tempo_resposta_medio': 0.9,
            'requisicoes_suspeitas': 15,
            'descricao': 'Muitas requisições suspeitas'
        },
        {
            'nome': 'Sistema Normal',
            'requisicoes_por_minuto': 150,
            'erros_por_minuto': 2,
            'tempo_resposta_medio': 0.7,
            'requisicoes_suspeitas': 1,
            'descricao': 'Métricas dentro do normal'
        }
    ]
    
    print("🔍 TESTE DE ANÁLISE DE OBSERVABILIDADE")
    print("=" * 60)
    print("\nAnalise cada cenário e identifique o problema:")
    print("   �� Ataque DDoS - Muitas requisições normais")
    print("   �� Bug no Sistema - Muitos erros")
    print("       🐌 Performance Lenta - Tempo de resposta alto")
    print("   🚨 Ataque de Segurança - Muitas requisições suspeitas")
    print("   ✅ Sistema Normal - Tudo funcionando bem")
    
    acertos = 0
    total = len(cenarios)
    
    for i, cenario in enumerate(cenarios, 1):
        print(f"\n{i}. Cenário: {cenario['nome']}")
        print(f"   �� Requisições/min: {cenario['requisicoes_por_minuto']}")
        print(f"   ❌ Erros/min: {cenario['erros_por_minuto']}")
        print(f"   ⏱️ Tempo resposta: {cenario['tempo_resposta_medio']}s")
        print(f"   🚨 Requisições suspeitas: {cenario['requisicoes_suspeitas']}")
        print(f"   📝 Descrição: {cenario['descricao']}")
        
        # Simulando análise automática
        if cenario['requisicoes_por_minuto'] > 400:
            analise_auto = "Ataque DDoS"
        elif cenario['erros_por_minuto'] > 20:
            analise_auto = "Bug no Sistema"
        elif cenario['tempo_resposta_medio'] > 3.0:
            analise_auto = "Performance Lenta"
        elif cenario['requisicoes_suspeitas'] > 10:
            analise_auto = "Ataque de Segurança"
        else:
            analise_auto = "Sistema Normal"
        
        print(f"   �� Análise Automática: {analise_auto}")
        
        # Verificando acerto
        if analise_auto == cenario['nome']:
            acertos += 1
            print(f"   ✅ CORRETO!")
        else:
            print(f"   ❌ INCORRETO! Análise correta: {cenario['nome']}")
    
    # Resultado final
    print(f"\n RESULTADO FINAL:")
    print(f"   Acertos: {acertos}/{total}")
    print(f"   Taxa de acerto: {(acertos/total)*100:.1f}%")
    
    if acertos >= total * 0.8:
        print("   🏆 EXCELENTE! Você tem um bom olho para análise de observabilidade!")
    elif acertos >= total * 0.6:
        print("   👍 BOM! Você está no caminho certo!")
    else:
        print("   Continue estudando! A prática leva à perfeição!")
    
    return acertos, total

# Executando o teste
acertos, total = testar_analise_observabilidade()

# Logging do resultado
logger.info(f"Teste de análise de observabilidade concluído: {acertos}/{total} acertos")

## **🏆 Desafio do Módulo - Sistema de Observabilidade Completo**

Agora é hora de colocar em prática! Vamos criar um sistema completo de observabilidade:

In [ ]:
# 🏆 DESAFIO DO MÓDULO - SISTEMA DE OBSERVABILIDADE COMPLETO

class SistemaObservabilidadeCompleto:
    def __init__(self):
        # TODO: Implemente as variáveis necessárias
        # - Sistema de métricas (Prometheus)
        # - Sistema de logs estruturados
        # - Sistema de alertas
        # - Dashboard em tempo real
        # - Análise de padrões
        pass
    
    def configurar_metricas(self):
        """Configura métricas do Prometheus"""
        # TODO: Implemente métricas
        # - Contadores de requisições
        # - Histogramas de tempo de resposta
        # - Gauges de recursos
        # - Métricas customizadas
        pass
    
    def configurar_logs(self):
        """Configura sistema de logs estruturados"""
        # TODO: Implemente logs
        # - Logs estruturados em JSON
        # - Diferentes níveis de log
        # - Rotação de logs
        # - Filtros de log
        pass
    
    def configurar_alertas(self):
        """Configura sistema de alertas"""
        # TODO: Implemente alertas
        # - Alertas baseados em métricas
        # - Diferentes níveis de severidade
        # - Notificações (email, Slack, etc.)
        # - Escalação de alertas
        pass
    
    def criar_dashboard(self):
        """Cria dashboard em tempo real"""
        # TODO: Implemente dashboard
        # - Gráficos em tempo real
        # - Métricas principais
        # - Alertas ativos
        # - Histórico de eventos
        pass
    
    def analisar_padroes(self):
        """Analisa padrões nos dados"""
        # TODO: Implemente análise
        # - Detecção de anomalias
        # - Análise de tendências
        # - Correlação de eventos
        # - Machine Learning para detecção
        pass

print(" DESAFIO: Implemente o sistema de observabilidade completo acima!")
print("💡 Dicas:")
print("   - Use Prometheus para métricas")
print("   - Implemente logs estruturados em JSON")
print("   - Configure alertas com diferentes níveis")
print("   - Crie dashboard com gráficos em tempo real")
print("   - Implemente análise de padrões e anomalias")
print("   - Use machine learning para detecção automática")
print("   - Configure notificações (email, Slack, etc.)")

## **📈 Dashboard de Observabilidade em Tempo Real**

Vamos criar um dashboard completo para monitoramento em tempo real:

In [ ]:
# 📈 DASHBOARD DE OBSERVABILIDADE EM TEMPO REAL

# Dados simulados de métricas em tempo real
dados_metricas = {
    'requisicoes_total': 15420,
    'requisicoes_por_minuto': 45,
    'erros_total': 127,
    'erros_por_minuto': 2,
    'tempo_resposta_medio': 0.8,
    'requisicoes_suspeitas': 8,
    'usuarios_ativos': 23,
    'cpu_uso': 45.2,
    'memoria_uso': 67.8,
    'alertas_ativos': 3
}

# Status do sistema
status_sistema = {
    'api_principal': '�� Online',
    'banco_dados': '�� Online',
    'cache_redis': '�� Online',
    'servico_ml': '🟡 Degradado',
    'monitoramento': '🟢 Online'
}

# Alertas ativos
alertas_ativos = [
    {
        'tipo': 'performance_lenta',
        'severidade': 'medio',
        'descricao': 'Tempo de resposta acima do normal',
        'timestamp': '2024-01-15 14:32:15'
    },
    {
        'tipo': 'requisicoes_suspeitas',
        'severidade': 'alto',
        'descricao': 'Muitas tentativas de acesso suspeito',
        'timestamp': '2024-01-15 14:28:42'
    },
    {
        'tipo': 'servico_ml_degradado',
        'severidade': 'baixo',
        'descricao': 'Serviço de ML com performance reduzida',
        'timestamp': '2024-01-15 14:15:30'
    }
]

# Criando dashboard
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)

# Título principal
fig.suptitle('DASHBOARD DE OBSERVABILIDADE - SISTEMA DE IA', fontsize=20, fontweight='bold')

# Métricas principais (2x2 grid)
ax1 = fig.add_subplot(gs[0, 0])
ax1.text(0.5, 0.8, f"{dados_metricas['requisicoes_por_minuto']}", ha='center', va='center', fontsize=24, fontweight='bold', color='#4ecdc4')
ax1.text(0.5, 0.5, 'REQUISIÇÕES/MIN', ha='center', va='center', fontsize=12)
ax1.text(0.5, 0.2, f"Total: {dados_metricas['requisicoes_total']:,}", ha='center', va='center', fontsize=10, color='gray')
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')

ax2 = fig.add_subplot(gs[0, 1])
ax2.text(0.5, 0.8, f"{dados_metricas['erros_por_minuto']}", ha='center', va='center', fontsize=24, fontweight='bold', color='#ff6b6b')
ax2.text(0.5, 0.5, 'ERROS/MIN', ha='center', va='center', fontsize=12)
ax2.text(0.5, 0.2, f"Total: {dados_metricas['erros_total']}", ha='center', va='center', fontsize=10, color='gray')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')

ax3 = fig.add_subplot(gs[0, 2])
ax3.text(0.5, 0.8, f"{dados_metricas['tempo_resposta_medio']:.1f}s", ha='center', va='center', fontsize=24, fontweight='bold', color='#feca57')
ax3.text(0.5, 0.5, 'TEMPO RESPOSTA', ha='center', va='center', fontsize=12)
ax3.text(0.5, 0.2, 'Médio', ha='center', va='center', fontsize=10, color='gray')
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')

ax4 = fig.add_subplot(gs[0, 3])
ax4.text(0.5, 0.8, f"{dados_metricas['usuarios_ativos']}", ha='center', va='center', fontsize=24, fontweight='bold', color='#96ceb4')
ax4.text(0.5, 0.5, 'USUÁRIOS ATIVOS', ha='center', va='center', fontsize=12)
ax4.text(0.5, 0.2, 'Conectados', ha='center', va='center', fontsize=10, color='gray')
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')

# Status dos serviços
ax5 = fig.add_subplot(gs[1, :2])
servicos = list(status_sistema.keys())
status = list(status_sistema.values())
cores = ['green' if '��' in s else 'orange' if '🟡' in s else 'red' for s in status]

bars = ax5.bar(servicos, [1]*len(servicos), color=cores, alpha=0.8)
ax5.set_title('Status dos Serviços', fontsize=14, fontweight='bold')
ax5.set_ylabel('Status')
ax5.set_ylim(0, 1.2)
ax5.tick_params(axis='x', rotation=45)

for i, (servico, status_text) in enumerate(zip(servicos, status)):
    ax5.text(i, 0.6, status_text, ha='center', va='center', fontweight='bold')

# Uso de recursos
ax6 = fig.add_subplot(gs[1, 2:])
recursos = ['CPU', 'Memória']
uso = [dados_metricas['cpu_uso'], dados_metricas['memoria_uso']]
cores_recursos = ['#ff6b6b' if u > 80 else '#feca57' if u > 60 else '#4ecdc4' for u in uso]

bars = ax6.bar(recursos, uso, color=cores_recursos, alpha=0.8)
ax6.set_title('Uso de Recursos (%)', fontsize=14, fontweight='bold')
ax6.set_ylabel('Uso (%)')
ax6.set_ylim(0, 100)

for bar, valor in zip(bars, uso):
    ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
             f'{valor:.1f}%', ha='center', va='bottom', fontweight='bold')

# Alertas ativos
ax7 = fig.add_subplot(gs[2:, :])
ax7.axis('off')

alertas_texto = "🚨 ALERTAS ATIVOS\n" + "="*50 + "\n"
for i, alerta in enumerate(alertas_ativos, 1):
    severidade_emoji = "��" if alerta['severidade'] == 'alto' else "��" if alerta['severidade'] == 'medio' else "🟢"
    alertas_texto += f"{i}. {severidade_emoji} {alerta['tipo'].replace('_', ' ').title()}\n"
    alertas_texto += f"   Severidade: {alerta['severidade'].upper()}\n"
    alertas_texto += f"   Descrição: {alerta['descricao']}\n"
    alertas_texto += f"   Hora: {alerta['timestamp']}\n\n"

ax7.text(0.05, 0.95, alertas_texto, transform=ax7.transAxes, fontsize=12, 
         verticalalignment='top', fontfamily='monospace', fontweight='bold')

plt.show()

print("📊 Dashboard de Observabilidade gerado com sucesso!")
print(f"🎯 {dados_metricas['alertas_ativos']} alertas ativos")
print(f"�� Sistema monitorado em tempo real!")

---

## ** Resumo do Módulo**

Neste módulo, aprendemos:

### **👁️ Conceitos de Observabilidade:**
- **O que é Observabilidade** - Visibilidade completa do sistema
- **Três pilares** - Métricas, Logs e Traces
- **Monitoramento em tempo real** - Detecção proativa de problemas
- **Alertas automáticos** - Notificação imediata de problemas

### **🛠️ Ferramentas Aprendidas:**
- Sistema de métricas com Prometheus
- Logs estruturados em JSON
- Sistema de alertas configurável
- Dashboard em tempo real
- Análise de padrões e anomalias

### **🔬 Técnicas Práticas:**
- Como configurar observabilidade
- Como analisar métricas em tempo real
- Como detectar problemas proativamente
- Como criar dashboards eficazes
- Como configurar alertas inteligentes

### ** Próximos Passos:**
- No próximo módulo, vamos aprender sobre **Detecção de Anomalias**
- Como usar Machine Learning para detectar comportamentos suspeitos
- Técnicas avançadas de detecção

---

**💡 Dica do Professor**: Observabilidade é como ter "olhos em todos os lugares" - você precisa ver tudo que está acontecendo para proteger seu sistema!

**🚀 Próximo módulo**: [Módulo 5: Detecção de Anomalias](./05_deteccao_anomalias.ipynb)

---

**🎉 Parabéns! Você completou o Módulo 4!** ��️

**🏆 Conquista Desbloqueada**: "Vigilante Digital" - Você agora pode monitorar sistemas de IA em tempo real!